# Solution for Question Number 1

Import required modules

In [ ]:
import os, re, json
import fitz

In [ ]:
directory = os.getcwd() + '\\..\\src_file'
file_name = '\\keppel-corporation-limited-annual-report-2018.pdf'
pdf = directory + file_name
# Initialize the directory of the file

In [ ]:
doc = fitz.open(pdf)
page_num = 11 #Initialize the page number here

page = doc.load_page(page_num)
# Page 13 of pdf stored to page variable

In [ ]:
blocks = page.get_text('json')
# Parse all blocks

pgdict = json.loads(blocks)
#Convert the blocks into json


def sort_blocks(block_dict):
    sortedBlocks = []
    for block in block_dict['blocks']:
        
        #Test if the block contains lines of text
        #Since images block does not have lines
        try:
            block['lines']
        except:
            pass
        else:
            x = block['bbox'][0]
            y = block['bbox'][1]
            key = [x, y]
            sortedBlocks.append([key, block])
        
    sortedBlocks.sort(key = lambda k: [k[0], k[1]])
    #Sorts the list based on x (k[0]) and then y(k[1])
    
    return[block[1] for block in sortedBlocks]
    #Returns the blocks without the keys
    
print(len(sort_blocks(pgdict)))
blocks_list = sort_blocks(pgdict)
    

In [ ]:
def list_paragraphs(blocks_list):
    paragraphs = []
    for block in blocks_list:
        textList = []
        for line in block['lines']:
            #Test if the line contains spans
            #If none: pass, if present: enter loop
            try:
                line['spans']
            except:
                pass
            else:
                for span in line['spans']:
                #Test if the span is a paragraph content using size, color, and font
                #Since headers, page numbers, etc. have different sizes, color, and font compared to paragraph text
                    if (span['size'] == 8.0 and span['color'] == 2236191 and span['font'] == 'Roboto-Light'):
                        #Stores the text if it is a paragraph content
                        ##.replace, .encode, and .decode were used to remove unicode characters
                        ##.replace was needed for \xa0 since there is no space in between texts
                        textList.append(span['text'].replace('\xa0', ' ').encode("ascii", "ignore").decode())
            #Store the text List to paragraphs
        paragraphs.append(textList)
    return paragraphs
paragraphs = list_paragraphs(blocks_list)
print(list_paragraphs(blocks_list)[6])

In [ ]:
for block in pgdict["blocks"]:
    print (block["number"])
    print (block["bbox"])

In [ ]:
print (pgdict["blocks"][5]['lines'][2]['spans'][0])
print (type(pgdict["blocks"][7]['lines'][1]['spans']))
#print (pgdict["blocks"][7]['lines'][0])
#print (pgdict["blocks"])
#print (blocks_list[0]['lines'][1]['bbox'][0])